In [5]:
import pandas as pd
import gc
from dask.distributed import Client
import dask.dataframe as dd


In [1]:
# Load the cleaned merged weather IATA data (ensure this is a CSV)
weather_iata_df = pd.read_csv(r"C:\Users\hopeh\Desktop\data_science_bootcamp\flight_times_capstone\weather_iata.csv")

# Load the cleaned merged flight IATA data (ensure this is a CSV)
flight_iata_df = pd.read_csv(r"C:\Users\hopeh\Desktop\data_science_bootcamp\flight_times_capstone\flights_iata.csv")


In [3]:
# Display the first few rows of each DataFrame to confirm they loaded correctly
print(weather_iata_df.head())

   latitude  longitude  elevation        date  prcp  snow  snwd  tmax  tmin  \
0     31.95     -85.13  86.800003  2024-01-01  0.00   NaN   NaN  65.0  32.0   
1     31.95     -85.13  86.800003  2024-01-02  0.00   NaN   NaN  55.0  27.0   
2     31.95     -85.13  86.800003  2024-01-03  0.14   NaN   NaN  43.0  27.0   
3     31.95     -85.13  86.800003  2024-01-04  0.00   NaN   NaN  58.0  31.0   
4     31.95     -85.13  86.800003  2024-01-05  0.23   NaN   NaN  61.0  29.0   

   tobs                          city state_abbr_x    state iata  \
0   NaN  eufaula weedon field airport           al  alabama  euf   
1   NaN  eufaula weedon field airport           al  alabama  euf   
2   NaN  eufaula weedon field airport           al  alabama  euf   
3   NaN  eufaula weedon field airport           al  alabama  euf   
4   NaN  eufaula weedon field airport           al  alabama  euf   

   airport_name state_abbr_y  
0  weedon field           al  
1  weedon field           al  
2  weedon field        

In [5]:
# Display the first few rows of each DataFrame to confirm they loaded correctly
print(flight_iata_df.head())

   year  month  day_of_month  day_of_week        date op_unique_carrier  \
0  2022      5             1            7  2022-05-01                9e   
1  2022      5             1            7  2022-05-01                9e   
2  2022      5             1            7  2022-05-01                9e   
3  2022      5             1            7  2022-05-01                9e   
4  2022      5             1            7  2022-05-01                9e   

  tail_num  op_carrier_fl_num origin_iata                origin_city  ...  \
0   n131ev               4633         atl                    atlanta  ...   
1   n131ev               4633         hpn               white plains  ...   
2   n131ev               4717         tys                  knoxville  ...   
3   n131ev               4958         atl                    atlanta  ...   
4   n131ev               4958         oaj  jacksonville/camp lejeune  ...   

                                       airport_name latitude  longitude  \
0  hartsfie

In [7]:
weather_iata_df.shape

(146022, 16)

In [9]:
flight_iata_df.shape

(12951297, 47)

In [15]:
print(weather_iata_df.columns)

Index(['latitude', 'longitude', 'elevation', 'date', 'prcp', 'snow', 'snwd',
       'tmax', 'tmin', 'tobs', 'city', 'state_abbr_x', 'iata', 'airport_name'],
      dtype='object')


In [17]:
print(flight_iata_df.columns)

Index(['year', 'month', 'day_of_month', 'day_of_week', 'date',
       'op_unique_carrier', 'tail_num', 'op_carrier_fl_num', 'origin_iata',
       'origin_city', 'dest_iata', 'dest_city', 'crs_dep_time', 'dep_time',
       'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in', 'crs_arr_time',
       'arr_time', 'cancelled', 'diverted', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'distance_group', 'carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay', 'origin_state', 'dest_state',
       'state', 'iata', 'airport_name', 'latitude', 'longitude', 'state_abbr',
       'state_dest', 'iata_dest', 'airport_name_dest', 'latitude_dest',
       'longitude_dest', 'state_abbr_dest'],
      dtype='object')


In [43]:
# Ensure the 'date' column is in datetime format
flight_iata_df['date'] = pd.to_datetime(flight_iata_df['date'])

# Define the date range
start_date = '2023-05-01'
end_date = '2024-04-30'

# Filter the DataFrame
flight_iata_filtered = flight_iata_df[(flight_iata_df['date'] >= start_date) & (flight_iata_df['date'] <= end_date)]

# Display the shape or first few rows of the filtered DataFrame
print("Filtered data shape:", flight_iata_filtered.shape)
print(flight_iata_filtered.head())

Filtered data shape: (6821609, 47)
         year  month  day_of_month  day_of_week       date op_unique_carrier  \
6129688  2023      5             1            1 2023-05-01                9e   
6129689  2023      5             1            1 2023-05-01                9e   
6129690  2023      5             1            1 2023-05-01                9e   
6129691  2023      5             1            1 2023-05-01                9e   
6129692  2023      5             1            1 2023-05-01                9e   

        tail_num  op_carrier_fl_num origin_iata  origin_city  ...  \
6129688   n131ev               4704         msp  minneapolis  ...   
6129689   n131ev               4888         cvg   cincinnati  ...   
6129690   n131ev               5128         dsm   des moines  ...   
6129691   n131ev               5190         msp  minneapolis  ...   
6129692   n131ev               5283         cvg   cincinnati  ...   

                                              airport_name   latitude

In [45]:
# Round latitude and longitude to two decimal places
weather_iata_df['latitude'] = weather_iata_df['latitude'].round(2)
weather_iata_df['longitude'] = weather_iata_df['longitude'].round(2)

flight_iata_df['latitude'] = flight_iata_df['latitude'].round(2)
flight_iata_df['longitude'] = flight_iata_df['longitude'].round(2)

In [47]:
# Sample sizes for testing
test_size_flights = 1000  # Adjust as necessary
test_size_weather = 1000   # Adjust as necessary

# Create smaller subsets for testing
small_flight_iata_df = flight_iata_df.sample(n=test_size_flights, random_state=1)
small_weather_iata_df = weather_iata_df.sample(n=test_size_weather, random_state=1)

# Merge for origin
merged_origin_test = pd.merge(
    small_flight_iata_df,
    small_weather_iata_df,
    left_on='origin_iata',
    right_on='iata',
    how='left',
    suffixes=('', '_origin')
)

# Merge for destination
final_merged_test = pd.merge(
    merged_origin_test,
    small_weather_iata_df,
    left_on='dest_iata',
    right_on='iata',
    how='left',
    suffixes=('', '_dest')
)

# Check the results
print("Test merged data shape:", final_merged_test.shape)
print("Sample of the merged data:")
print(final_merged_test.head())

Test merged data shape: (1509, 79)
Sample of the merged data:
   year  month  day_of_month  day_of_week       date op_unique_carrier  \
0  2022      8            11            4 2022-08-11                ha   
1  2023      9            22            5 2023-09-22                yx   
2  2023      2             7            2 2023-02-07                wn   
3  2022     12             8            4 2022-12-08                ua   
4  2024      4            16            2 2024-04-16                aa   

  tail_num  op_carrier_fl_num origin_iata        origin_city  ... snwd_dest  \
0   n483ha                548         hnl           honolulu  ...       NaN   
1   n207jq               5782         cmh           columbus  ...       NaN   
2   n8657b               1481         jax       jacksonville  ...       NaN   
3   n848ua               1063         tvc      traverse city  ...       NaN   
4   n107us               2352         dfw  dallas/fort worth  ...       NaN   

  tmax_dest  tmin_

In [48]:
print(final_merged_test.columns)

Index(['year', 'month', 'day_of_month', 'day_of_week', 'date',
       'op_unique_carrier', 'tail_num', 'op_carrier_fl_num', 'origin_iata',
       'origin_city', 'dest_iata', 'dest_city', 'crs_dep_time', 'dep_time',
       'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in', 'crs_arr_time',
       'arr_time', 'cancelled', 'diverted', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'distance_group', 'carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay', 'origin_state', 'dest_state',
       'state', 'iata', 'airport_name', 'latitude', 'longitude', 'state_abbr',
       'state_dest', 'iata_dest', 'airport_name_dest', 'latitude_dest',
       'longitude_dest', 'state_abbr_dest', 'latitude_origin',
       'longitude_origin', 'elevation', 'date_origin', 'prcp', 'snow', 'snwd',
       'tmax', 'tmin', 'tobs', 'city', 'state_abbr_x', 'state_origin',
       'iata_origin', 'airport_name_origin', 'state_abbr_y', 'latitud

In [51]:
# Drop the specified columns from weather_iata_df
#weather_iata_df = weather_iata_df.drop(columns=[])

In [53]:
# optimize dataframe
def optimize_dataframe(df):
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    for col in df.select_dtypes(include=['object']).columns:
        if df[col].nunique() / len(df) < 0.5:  # Adjust threshold as needed
            df[col] = df[col].astype('category')
    return df

chunksize = 25000  # Adjust based on memory capacity

# Process flight_iata_df in chunks
optimized_flight_chunks = []
for chunk in pd.read_csv(r'C:\Users\hopeh\Desktop\data_science_bootcamp\flight_times_capstone\flights_iata.csv', chunksize=chunksize):
    optimized_chunk = optimize_dataframe(chunk)
    optimized_flight_chunks.append(optimized_chunk)
    del chunk  # Delete the chunk to free up memory
    gc.collect()  # Trigger garbage collection

# Concatenate all processed flight chunks into a single DataFrame
flight_iata_df = pd.concat(optimized_flight_chunks, ignore_index=True)

# Process weather_iata_df in chunks
optimized_weather_chunks = []
for chunk in pd.read_csv(r'C:\Users\hopeh\Desktop\data_science_bootcamp\flight_times_capstone\weather_iata.csv', chunksize=chunksize):
    optimized_chunk = optimize_dataframe(chunk)
    optimized_weather_chunks.append(optimized_chunk)
    del chunk  # Delete the chunk to free up memory
    gc.collect()  # Trigger garbage collection

# Concatenate all processed weather chunks into a single DataFrame
weather_iata_df = pd.concat(optimized_weather_chunks, ignore_index=True)

# Now flight_iata_df and weather_iata_df are optimized


In [60]:
# def merge_flights_with_weather(flight_iata_df, weather_iata_df, chunk_size=500):
#     # Temporary file to store the merged data
#     output_file = 'merged_data.csv'
    
#     total_chunks = len(flight_iata_df) // chunk_size + 1  # Calculate total chunks

#     # Process flight_iata_df in chunks
#     for i in range(total_chunks):
#         flight_chunk = flight_iata_df.iloc[i * chunk_size:(i + 1) * chunk_size]  # Create a chunk

#         # Merge for origin
#         merged_origin_chunk = pd.merge(
#             flight_chunk,
#             weather_iata_df,
#             left_on='origin_iata',
#             right_on='iata',
#             how='left',
#             suffixes=('', '_origin')
#         )

#         # Merge for destination
#         merged_final_chunk = pd.merge(
#             merged_origin_chunk,
#             weather_iata_df,
#             left_on='dest_iata',
#             right_on='iata',
#             how='left',
#             suffixes=('', '_dest')
#         )

#         # Append the merged chunk to a CSV file
#         merged_final_chunk.to_csv(output_file, mode='a', header=(i==0), index=False)

#         # Print progress
#         print(f"Processing chunk {i + 1}/{total_chunks}")

#         # Clean up
#         del flight_chunk, merged_origin_chunk, merged_final_chunk
#         gc.collect()  # Trigger garbage collection

#     # Read the merged data back into a DataFrame if needed
#     final_merged_data = pd.read_csv(output_file)
#     return final_merged_data

# # Call the function with your DataFrames
# final_merged_data = merge_flights_with_weather(flight_iata_df, weather_iata_df)
# print("Merged data shape:", final_merged_data.shape)

Processing chunk 1/25903


KeyboardInterrupt: 

In [11]:
# Initialize the Dask client with specific settings
client = Client(n_workers=4, threads_per_worker=1, port=8788, nanny=False)

def merge_flights_with_weather_dask(flight_file, weather_file):
    # Load data as Dask DataFrames
    flight_iata_dd = dd.read_csv(flight_file)
    weather_iata_dd = dd.read_csv(weather_file)

    # Merge for origin
    merged_origin_dd = dd.merge(
        flight_iata_dd,
        weather_iata_dd,
        left_on='origin_iata',
        right_on='iata',
        how='left',
        suffixes=('', '_origin')
    )

    # Merge for destination
    merged_final_dd = dd.merge(
        merged_origin_dd,
        weather_iata_dd,
        left_on='dest_iata',
        right_on='iata',
        how='left',
        suffixes=('', '_dest')
    )

    # Compute the final DataFrame
    final_merged_data = merged_final_dd.compute()
    
    return final_merged_data

# Call the function with the paths to your CSV files
flight_file = r'C:\Users\hopeh\Desktop\data_science_bootcamp\flight_times_capstone\flights_iata.csv'
weather_file = r'C:\Users\hopeh\Desktop\data_science_bootcamp\flight_times_capstone\weather_iata.csv'
final_merged_data = merge_flights_with_weather_dask(flight_file, weather_file)

print("Merged data shape:", final_merged_data.shape)


C:\Users\hopeh\anaconda3\Lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 50350 instead
  warnings.warn(


RuntimeError: Nanny failed to start.

In [ ]:
print(final_merged_data.head())